In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import classification_report

# Veriyi yükle
df = pd.read_csv("/content/Customer-Churn-Records.csv")

# Gereksiz sütunları çıkar
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

# Hedef ve özellikleri ayır
X = df.drop('Exited', axis=1)
y = df['Exited']

# Kategorik verileri one-hot encode et
categorical = ['Geography', 'Gender', 'Card Type']
ct = ColumnTransformer([("encoder", OneHotEncoder(drop='first'), categorical)], remainder='passthrough')
X = ct.fit_transform(X)

# Veriyi ölçekle
sc = StandardScaler()
X = sc.fit_transform(X)

# PCA uygula
pca = PCA(n_components=0.95, random_state=42)
X = pca.fit_transform(X)

# Eğitim/test ayır
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ANN modeli kur
model = Sequential()
model.add(Dense(16, activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5, batch_size=32, validation_split=0.2, verbose=1)

# Sonuçlar
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

print("SONUÇLAR:")
print(classification_report(y_test, y_pred))


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Veriyi oku
df = pd.read_csv("/content/Customer-Churn-Records.csv")

# Gereksiz sütunları kaldır
df = df.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1)

# Kategorik verileri one-hot encode et
df_encoded = pd.get_dummies(df, drop_first=True)

# Korelasyon matrisini hesapla
corr_matrix = df_encoded.drop('Exited', axis=1).corr()

# Isı haritasını çiz
plt.figure(figsize=(14, 10))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", cbar=True)
plt.title("Feature'ların Birbirleriyle Olan Korelasyon Matrisi", fontsize=14)
plt.xticks(rotation=45)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Veri yükle
df = pd.read_csv("/content/Customer-Churn-Records.csv")

# Yaş gruplarını oluştur
df['AgeGroup'] = pd.cut(df['Age'], bins=[18, 30, 40, 50, 60, 100],
                        labels=['18-30', '31-40', '41-50', '51-60', '60+'])

# Her yaş grubundaki Exited ve Not Exited oranlarını hesapla
age_exit_dist = df.groupby(['AgeGroup', 'Exited']).size().unstack().fillna(0)
age_exit_ratio = age_exit_dist.div(age_exit_dist.sum(axis=1), axis=0)

# Görselleştir
ax = age_exit_ratio.plot(kind='bar', stacked=False, figsize=(8, 6), color=['skyblue', 'salmon'])
plt.title("Yaş Gruplarına Göre Exited ve Not Exited Oranları", fontsize=14)
plt.xlabel("Yaş Grubu")
plt.ylabel("Oran")
plt.legend(["Not Exited (0)", "Exited (1)"])
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.tight_layout()
plt.show()
